In [1]:
import sys
import os
print(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.getcwd()))

/home/ferdydh/tum-adlr-05


In [2]:
from scripts import initialize

initialize()

Seed set to 42


In [3]:
import json
import os

# Read the list of used files from JSON
with open("../logs/checkpoints/run_20250202_233846/used_grasp_files.json", 'r') as f:
    used_files = json.load(f)

# Get all files in the grasps directory
all_files = os.listdir("../data/grasps")

# Find files that are not in the used_files list
unused_files = [file for file in all_files if file not in used_files]

# Print the unused files
print("Files not used in training:")
for file in unused_files:
    print(file)

Files not used in training:
AccentTable_828d1b43814fc98c93b89bfd41acaad0_0.005164979590317853.h5
Couch_7c6491c7fe65449cd9bdef3310c0e5a6_0.0012915097905271376.h5
Bear_a200c5af60e49857b7d0eee5f296f7bb_0.011823037163955013.h5
PersonStanding_f741864eb54d47d466da45b121c8ad2d_0.0031158197641377354.h5
Sword_1018f01d42ae7fad52249d8432f6087e_0.002415045419046423.h5
CerealBox_27f2de454f50ccea39f3410d5f76299b_0.010866565625042176.h5
CoffeeTable_3250cee2b00f2bfc9141594c44ac35a8_0.0013663476120025306.h5
Banana_ba0d56295321002718ddbf38fa69c501_0.009459827629638647.h5
Pillow_9d183dcd7261cfffe6e130a30b5ed875_0.00436549667883951.h5
Pen_f234b3f39edb7f1d8ef19a48ac4fcfdf_0.0018716592484421207.h5
Barstool_be9f6f2ff5f9e1f5d810b14a81e12eca_0.004595050772114898.h5
Shower_f14d5177892763576d4141a2a3994e3e_0.008273107292394586.h5
RoundTable_4a8342bae986e48e504721639e19f609_5.8606349840755256e-05.h5
Basket_a124998e1cf5395b924bf081da6f024c_0.005748400938563568.h5
LightSwitch_ac9728f9b72fca00b48b2b84c76738a_0.05875

In [4]:
print(len(unused_files))
print(len(used_files))
print(len(all_files))

836
8000
8836


In [5]:
from src.models.lightning import Lightning
model = Lightning.load_from_checkpoint("../logs/checkpoints/run_20250202_233846/last.ckpt")

In [6]:
model.eval()

Lightning(
  (model): VelocityNetwork(
    (sdf_encoder): VoxelSDFEncoder(
      (encoder): Sequential(
        (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): ReLU()
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (4): ReLU()
        (5): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (7): ReLU()
        (8): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (9): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (10): ReLU()
        (11): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (avg_pool): AdaptiveAvgPool3d(output_size=(1, 1, 1))
    )
    (time_proj): Sequential(
      

In [7]:
from data.dataset import DataModule
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl
import wandb

from src.core.config import DataConfig, ExperimentConfig

config: ExperimentConfig = ExperimentConfig.default_mlp()
config.data.sample_limit = None
config.data.files = unused_files
config.data.batch_size = 128
config.data.dataset_workers = 15
config.data.data_path = "../data"


wandb.finish()
# Initialize WandB logger with modified settings
wandb_logger = WandbLogger(
    project=config.training.project_name,
    name=config.training.run_name,
    save_dir=config.training.save_dir,
    settings=wandb.Settings(start_method="thread"),
)

# Setup callbacks
callbacks = []

# Initialize trainer
trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=callbacks,
    accelerator="auto",
    devices="auto",
    log_every_n_steps=1,
)


data = DataModule(config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [8]:
trainer.test(model, datamodule=data)

wandb: Currently logged in as: ferdydh (ferdydh-technical-university-of-munich). Use `wandb login --relogin` to force relogin
wandb: WARNING Path logs/wandb/ wasn't writable, using system temp directory.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:data.dataset:Number of .h5 files to process: 836
INFO:data.dataset:Global min: [-0.30063249 -0.27909944 -0.3358168 ], max: [0.28618389 0.28372441 0.33662377]

Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined